In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
from tqdm import trange
import sys

Lambda_idx = 0
nbins = 5



Lambda = Lambdas[Lambda_idx]


# CCAT-prime
zmin = 3.5
zmax = 8.1

Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

chi_bin_edges = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), nbins + 1)
chi_bin_centers = (chi_bin_edges[1:] + chi_bin_edges[:-1])/2
dchi_binned = np.mean(np.diff(chi_bin_edges))


oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/I_ILo_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_nbins_%d_cubature.npy'%(zmin, zmax, Lambda_idx, nbins)
print(oup_fname)

Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

from  LIMxCMBL.kernels import *
f_KLIM   = get_f_KI()

f_KLIM_windowed = apply_window(f_K = f_KLIM,
                               chimin = chimin,
                               chimax = chimax)

from scipy.interpolate import interp1d, interp2d, LinearNDInterpolator

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')

from scipy.integrate import quad_vec#, cubature

tmp_chibs = []
tmp_log_deltas = []
tmp_fnctn = []
for i in range(len(chibs)):
    for j in range(len(deltas)):
        tmp_chibs += [chibs[i]]
        tmp_log_deltas += [np.log10(deltas[j])]
        tmp_fnctn += [inner_dkparp_integral[:,i,j]]
        
f_inner_integral = LinearNDInterpolator(list(zip(tmp_chibs, tmp_log_deltas)), tmp_fnctn)

/scratch/users/delon/LIMxCMBL/I_auto/I_ILo_zmin_3.5_zmax_8.1_Lambda_idx_0_nbins_5_cubature.npy


In [2]:
#cant cubature in notebook because sherlock is weird, but can run py3.13 in terminal
#will just make sure integrand has correct properties

In [107]:

def integrand(x):
    '''
    x should be of shape (npoints, ndim)
    output should be of shape (npoints, output_dim_1,...)
    '''
    p = len(x)
    chi  = x[:,0]
    chips = x[:,1:nbins+1]
    _chib = x[:,-1]
    
    
    _delta = np.abs(1 - chi, 1/_chib)
    _delta = np.where(_delta < 1e-6, 1e-6,
                     np.where(_delta > 0.7, 0.7,
                              _delta))
          
    integrand = 2 * f_KLIM_windowed(chi)
    print(integrand.shape)
    integrand = np.einsum('p,pl->pl',
                          integrand,
                          f_inner_integral(list(zip(_chib, np.log10(_delta)))))

    integrand = np.einsum('p,pl->pl', 1/(_chib**2), integrand)
    
    f_KLIMLo = get_f_KILo(external_chi = chips, Lambda=Lambda)

    f_KLIMLo_windowed = apply_window(f_K = f_KLIMLo,
                                     chimin = chimin,
                                     chimax = chimax)

    kernel = f_KLIMLo_windowed((2*_chib.reshape(-1,1) - chi.reshape(p,1)))
    return np.einsum('pj, pl->plj', kernel, integrand)

In [112]:
inp = np.array([np.hstack([np.random.uniform(chimin, chimax), chi_bin_centers, np.random.uniform(chimin, chimax)]) for _ in range(1000000)])

In [113]:
print(inp[:3,0])

[7430.46225215 8228.17820808 8082.73753129]


In [114]:
print(integrand(inp)[:3, 0])

(1000000,)
[[-1.11277220e-19 -1.11278859e-19 -1.11279773e-19 -1.11279963e-19
  -1.11279428e-19]
 [-4.28930425e-20 -4.28935372e-20 -4.28937528e-20 -4.28936891e-20
  -4.28933462e-20]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]


In [115]:
import cProfile
cProfile.run('integrand(inp)')

(1000000,)
         390 function calls in 4.175 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.440    1.440    4.018    4.018 1855438159.py:1(integrand)
        1    0.157    0.157    4.175    4.175 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 _base.py:1513(issparse)
        6    0.000    0.000    0.000    0.000 _interpolate.py:381(_do_extrapolate)
        3    0.000    0.000    0.001    0.000 _interpolate.py:496(__init__)
        3    0.000    0.000    0.000    0.000 _interpolate.py:649(fill_value)
        3    0.000    0.000    0.000    0.000 _interpolate.py:680(_check_and_update_bounds_error_for_extrapolation)
        4    0.062    0.016    0.416    0.104 _interpolate.py:690(_call_linear)
        4    0.000    0.000    0.416    0.104 _interpolate.py:759(_evaluate)
        4    0.004    0.001    0.004    0.001 _methods.py:90(_clip)
        3    0.000    0.000    0.000    0.000 _polyin